In [1]:
import sevenbridges as sbg
from sevenbridges.errors import SbgError
from sevenbridges.http.error_handlers import rate_limit_sleeper, maintenance_sleeper
import sys
import re
import pdb
import concurrent.futures
from requests import request
import json
config = sbg.Config(profile='turbo')
api = sbg.Api(config=config, error_handlers=[rate_limit_sleeper, maintenance_sleeper])

## Set up tasks

In [10]:
def get_refs(project):
    ref_objs = {}
    ref_objs['reference_genome'] = api.files.query(project=project, names=['Homo_sapiens_assembly38.fasta'])[0]
    ref_objs['normalize_flag'] = True
    return ref_objs

In [11]:
def draft_tasks(project, bs_id, ref_dict, app_name):
    input_dict = {}
    for key in ref_dict:
        input_dict[key] = ref_dict[key]
    input_dict['input_vcf'] = inputs[bs_id]
    task_name = "pyspark vcf2parquet: " + bs_id
    task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=input_dict, run=False)
    task.inputs['output_basename'] = bs_id
    task.save()


In [12]:
gvcf_manifest = open('/Users/brownm28/Documents/2022-Feb-7_vcf2parquet/manifest_20220207_162148.tsv')
project = 'd3b-bixu/kf-gvcf-merge-dev'
app_name = project + '/pyspark_vcf2parquet'
ref_dict = get_refs(project)
inputs = {}
head = next(gvcf_manifest)
# get field position of sample ID
header = head.rstrip('\n').split('\t')
s_idx = header.index('sample_id')
for line in gvcf_manifest:
    info = line.rstrip('\n').split('\t')
    gvcf_obj = api.files.get(info[0])
    inputs[info[s_idx]] = gvcf_obj
gvcf_manifest.close()
for bs_id in inputs:
    draft_tasks(project, bs_id, ref_dict, app_name)


## Run tasks

In [13]:
project = 'd3b-bixu/kf-gvcf-merge-dev'
tasks = api.tasks.query(project=project, status="DRAFT")
phrase = 'pyspark vcf2parquet'
for task in tasks:
    if phrase in task.name:
        task.run()

    